In [1]:
import pytorch_lightning as pl

/Users/philippkiesling/.conda/envs/linkit_hand_gesture_recognition/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
